<a href="https://colab.research.google.com/github/xtt001/LLM-Learning/blob/main/resume_parse_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install openai

In [11]:
from openai import OpenAI
import json  # 引入json库，虽然还没用到解析，先习惯带上

# 记得替换你的 Key 和 Host
client = OpenAI(
    api_key="sk-xxxxxxxxxxxxxxxxx",
    base_url="https://llmapi.paratera.com"
)
model_name = "Qwen3-235B-A22B-Instruct-2507"

In [5]:

# 1. 定义一段乱七八糟的简历文本 (模拟用户输入)
raw_resume = """
我叫张三，电话是13800000000。我之前在字节跳动干了3年后端开发，主要用Python和Go。
后来去了腾讯做了一年产品经理。我现在想找一份AI开发的工作。
"""

# 2. 编写 System Prompt (核心！Prompt工程的体现)
# 注意看我怎么要求它的格式
system_prompt = """
你是一个简历解析助手。请从用户的简历中提取关键信息。
必须严格输出 JSON 格式，不要包含任何 markdown 标记或其他废话。
需要提取的字段包括：
- name (姓名)
- phone (电话)
- skills (技能列表，数组格式)
- experience_years (工作年限，数字)
- target_job (目标岗位)
"""

print("正在提取简历信息...")

response = client.chat.completions.create(
    model="Qwen3-235B-A22B-Instruct-2507", # 换成你测试通过的模型名
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": raw_resume}
    ],
    temperature=0.1  # 设为低温度，让它严谨，不要瞎编
)

result = response.choices[0].message.content
print("提取结果：")
print(result)

正在提取简历信息...
提取结果：
{
  "name": "张三",
  "phone": "13800000000",
  "skills": ["Python", "Go"],
  "experience_years": 4,
  "target_job": "AI开发"
}


In [14]:
def parse_resume(text):
    # 1. 定义一段乱七八糟的简历文本 (模拟用户输入)
    """
    我叫张三，电话是13800000000。我之前在字节跳动干了3年后端开发，主要用Python和Go。
    后来去了腾讯做了一年产品经理。我现在想找一份AI开发的工作。
    """
    raw_resume = text

    # 2. 编写 System Prompt (核心！Prompt工程的体现)
    # 注意看我怎么要求它的格式
    system_prompt = """
    你是一个简历解析助手。请从用户的简历中提取关键信息。
    必须严格输出 JSON 格式，不要包含任何 markdown 标记或其他废话。
    需要提取的字段包括：
    - name (姓名)
    - phone (电话)
    - skills (技能列表，数组格式)
    - experience_years (工作年限，数字)
    - target_job (目标岗位)
    """

    print("正在提取简历信息...")

    response = client.chat.completions.create(
        model=model_name, # 换成你测试通过的模型名
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": raw_resume}
        ],
        temperature=0.1  # 设为低温度，让它严谨，不要瞎编
    )

    result = response.choices[0].message.content
    return result

def critique_resume(text):
    # 1. 定义毒舌 HR 的人设
    hr_prompt = """
    你是一位拥有 20 年经验的资深 HR 总监，说话风格犀利、直击痛点。
    请阅读用户的简历，指出 3 个致命缺点，并给出具体的修改建议。
    请用 Markdown 格式输出，用加粗字体强调重点。注意控制篇幅尽量简洁
    """
    raw_resume = text

    print("HR 正在看简历...")

    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": hr_prompt},
            {"role": "user", "content": raw_resume} # 还是用上面那个张三的简历
        ],
        temperature=0.7 # 稍微调高一点，让它发挥文采
    )

    result = response.choices[0].message.content
    return result


# 主程序
if __name__ == "__main__":
    # 1. 让用户输入简历
    user_text = input("请输入你的简历内容：")

    # 2. 调用函数
    info = parse_resume(user_text)
    print(f"\n--- 提取的基本信息 ---\n{info}")

    review = critique_resume(user_text)
    print(f"\n--- HR 点评建议 ---\n{review}")

请输入你的简历内容：    我叫张三，电话是13800000000。我之前在字节跳动干了3年后端开发，主要用Python和Go。     后来去了腾讯做了一年产品经理。我现在想找一份AI开发的工作。
正在提取简历信息...

--- 提取的基本信息 ---
{
  "name": "张三",
  "phone": "13800000000",
  "skills": ["Python", "Go"],
  "experience_years": 4,
  "target_job": "AI开发"
}
HR 正在看简历...

--- HR 点评建议 ---
**简历致命缺点与修改建议**

1. **信息严重缺失，毫无专业性可言**  
   你只写了三句话，**连基本简历结构都没有**：无教育背景、无项目经历、无成果量化。HR 3 秒内就会扔进废件堆。  
   ✅ 修改建议：**必须补充完整模块**——教育背景、工作经历（公司、职位、时间）、核心职责、关键成果（用数据说话）。

2. **职业路径混乱，缺乏逻辑主线**  
   从“后端开发”跳“产品经理”，现在又要转“AI开发”，**转型动机完全未说明**，显得随波逐流。HR 会质疑你的稳定性和专业积累。  
   ✅ 修改建议：**在求职信和简历摘要中清晰阐述转型逻辑**，例如：“基于后端经验深入理解系统架构，结合产品视角，转向AI工程化落地”。

3. **技能与目标岗位严重脱节，空洞无支撑**  
   说想做 AI 开发，但**没提任何相关技术栈（如TensorFlow、PyTorch）或项目经验**，Python 虽有关联，但未突出AI应用场景。  
   ✅ 修改建议：**明确列出AI相关技能，并补充至少1-2个实战项目**（如“基于PyTorch的文本分类模型开发”），即使非工作经历也要写上。

> **总结：你现在交的不是简历，是一张便条。**  
> **立刻重构为结构化文档，聚焦AI岗位需求，用事实和数据建立可信度。**
